# **Sequence Classification**
Sequence classification is a natural language processing task of assigning a label or class to a given text.

We shall fine-tune a BERT model using TensorFlow to identify if a given two sentences are paraphrases or not (i.e., if both sentences mean the same thing) using MRPC (Microsoft Research Paraphrase Corpus) dataset.

### **1. Install and Import Required Libraries**

In [ ]:
!pip install datasets transformers evaluate

In [ ]:
import tensorflow as tf
import numpy as np

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
from evaluate import load

### **2. Load Data**

In [ ]:
raw_dataset = load_dataset('glue', 'mrpc')

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

### **3. Preprocess Data**

In [ ]:
model_checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenizer_function(example):
  return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

tokenized_dataset = raw_dataset.map(tokenizer_function, batched=True)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
batch_size = 8
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

tf_train_dataset = tokenized_dataset['train'].to_tf_dataset(
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    collate_fn = data_collator,
    shuffle = True,
    batch_size = batch_size
)

tf_validation_dataset = tokenized_dataset['validation'].to_tf_dataset(
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    collate_fn = data_collator,
    shuffle = False,
    batch_size = batch_size
)

tf_test_dataset = tokenized_dataset['test'].to_tf_dataset(
    columns = ['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols = ['label'],
    collate_fn = data_collator,
    shuffle = False,
    batch_size = batch_size
)

### **4. Define and Fine-tune the Model**

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

In [ ]:
num_epochs = 5
num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
history = model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs, verbose=1)

Epoch 1/5
459/459 [==============================] - 162s 212ms/step - loss: 0.6052 - accuracy: 0.6919 - val_loss: 0.5469 - val_accuracy: 0.7230
Epoch 2/5
459/459 [==============================] - 75s 164ms/step - loss: 0.4261 - accuracy: 0.8146 - val_loss: 0.3581 - val_accuracy: 0.8456
Epoch 3/5
459/459 [==============================] - 72s 157ms/step - loss: 0.1890 - accuracy: 0.9310 - val_loss: 0.4209 - val_accuracy: 0.8431
Epoch 4/5
459/459 [==============================] - 72s 156ms/step - loss: 0.0573 - accuracy: 0.9806 - val_loss: 0.6059 - val_accuracy: 0.8456
Epoch 5/5
459/459 [==============================] - 75s 163ms/step - loss: 0.0218 - accuracy: 0.9932 - val_loss: 0.6497 - val_accuracy: 0.8529


### **5. Predict using the Fine-tuned Model**



In [ ]:
predictions = model.predict(tf_test_dataset)['logits']
class_preds = np.argmax(predictions, axis=-1)
class_preds

216/216 [==============================] - 16s 58ms/step


array([1, 1, 1, ..., 1, 1, 1])

### **6. Compute Metrics**

In [ ]:
metric = load('glue', 'mrpc')

In [ ]:
metric.compute(predictions=class_preds, references=tokenized_dataset['test']['label'])

{'accuracy': 0.8394202898550724, 'f1': 0.8834665544804374}